In [56]:
from collections import defaultdict
from dataclasses import dataclass

from Bio import SeqIO

import plotly.graph_objects as go
import networkx as nx

In [62]:
e = Edge(1, 2)

a = set()
a.add(e)

# New heading

In [319]:
@dataclass
class Edge:
    source: int
    sink: int
    coverage: float = 0
    seq: str = ""
        
    def __hash__(self):
        return hash(self.seq)
    
class DeBruijnGraph:
    def __init__(self, k=55):
        self.k = k
        self.label = ''  # название графа при отрисовке
        
        self.adj_list_out = defaultdict(lambda: defaultdict(set))
        self.adj_list_in = defaultdict(lambda: defaultdict(set))
        self.indegree = defaultdict(int)  # чтобы ловить разветвления
        
        self.kmer_counter = 0
        self.kmer_mapping = {} # k-мер -> число 
        self.kmer_decode = {}  # число -> к-мер
        self.kmer_coverage = defaultdict(int)
        
    def add_node(self, kmer):
        if kmer not in self.kmer_mapping.keys():
            self.kmer_mapping[kmer] = self.kmer_counter
            self.kmer_decode[self.kmer_counter] = kmer
            self.kmer_counter += 1
        self.kmer_coverage[self.kmer_mapping[kmer]] += 1
    
    def add_edge(self, edge):
        source = edge[:-1]
        sink = edge[1:]
        
        self.add_node(source)
        self.add_node(sink)
        
        source_code = self.kmer_mapping[source]
        sink_code = self.kmer_mapping[sink]
        
        new_edge = Edge(source=source_code, sink=sink_code, seq=edge)
        
        if new_edge not in self.adj_list_out[source_code][sink_code]:
            
            self.adj_list_out[source_code][sink_code].add(new_edge)
            self.indegree[sink_code] += 1
        # self.adj_list_out[source_code][sink_code].add(new_edge)
        # self.indegree[sink_code] += 1
            self.adj_list_in[sink_code][source_code].add(new_edge)
        
    def read_file(self, path, extension='fasta'):
        """Adds k-mers (as nodes) and (k+1 mers) as edges from .fasta file"""
        if self.label != '':
            self.label += '; '
        self.label += path
        
        with open(path, 'r') as f:
            for rec in SeqIO.parse(f, extension):
                edge_len = self.k + 1
                for i in range(len(rec.seq) - edge_len + 1):
                    edge_seq = str(rec.seq[i:i+edge_len])
                    self.add_edge(edge_seq)
                    
    def euler_walk(self, source, sink, visited, edge_seq=""):
        visited[sink] = True
        # Если входащая и исходящая степень вершины равны единице, и мы в ней ещё не были (т.е. это 
        # не цикл), то продолжаем наращивать ребро; иначе - возвращаем 
        # текущую последовательность и вершину в качестве нового большого ребра
        if len(self.adj_list_out[sink]) == 1 and self.indegree[sink] == 1:
                next_sink = next(iter(self.adj_list_out[sink]))
                add_edge = next(iter(self.adj_list_out[sink][next_sink])).seq[self.k:] 
                edge_seq += next(iter(self.adj_list_out[sink][next_sink])).seq[self.k:] 
                self.adj_list_out.pop(sink)
                if not visited[next_sink]:
                    return self.euler_walk(sink, next_sink, visited, edge_seq)
                else:
                    return sink, edge_seq
        else:
            
            return sink, edge_seq
        
    def compress_edges(self):
        # """Compresses edges when no cycles or forks are present"""
        # first_iteration = True
        # while not self.is_compressed():
            if True: # first_iteration:
                first_iteration = False
                visited = {i: False for i in range(self.kmer_counter)}
            else:
                nodes = self.current_nodes()
                visited = {node: False for node in nodes}
                
            for i in visited.keys():
                if not visited[i]: 
                    edge_seq = self.kmer_decode[i]
                    for sink in list(self.adj_list_out[i].keys()):
                        if len(self.adj_list_out[i][sink]) == 1:
                            tmp = len(self.adj_list_out[i][sink])
                            assert tmp == 1
                            # starting_seq = next(iter(self.adj_list_out[i].pop(sink))).seq
                            new_sink, new_seq = self.euler_walk(i, sink, visited, self.kmer_decode[i])
                            new_edge = Edge(source=i, sink=new_sink, 
                                coverage=self.mean_seq_coverage(new_seq), seq=new_seq        
                            )
                            self.adj_list_out[i][new_sink].add(new_edge)
                if all(visited.values()):
                    break
        
                # подчистим всякое...
            for sink in list(self.adj_list_out.keys()):
                if len(self.adj_list_out[sink]) == 0:
                    self.adj_list_out.pop(sink)
                
    def is_compressed(self):
        # проверим, остались ли вершины со степенями 1
        nodes = self.current_nodes()
        return all(self.__check_degrees(node) for node in nodes)
       
    def current_nodes(self):
        """Возвращает множество вершин, сохранившихся в графе после сжатия"""
        nodes = set()
        for source, sinks in self.adj_list_out.items():
            nodes.add(source)
            for sink in sinks:
                nodes.add(sink)
        return nodes
    
    def __check_degrees(self, node):
        # проверяет степени вершины
        if len(self.adj_list_out[node]) == 1 and self.indegree[node] == 1:
            return False
        return True
                
    def mean_seq_coverage(self, seq):
            total = 0
            N = len(seq) - self.k + 1
            for i in range(N):
                kmer = seq[i:i+self.k]
                code = self.kmer_mapping[kmer]
                total += self.kmer_coverage[code]
            return total / N
    
    def make_dot_graph(self):
        dot = Digraph(comment=self.label)
        for source, sinks in self.adj_list_out.items():
            dot.node(str(source))
            for sink in sinks:
                dot.node(str(sink))
                for edge in self.adj_list_out[source][sink]:
                    # print(edge)
                    if edge.coverage == 0:
                        edge.coverage = self.mean_seq_coverage(edge.seq)
                    dot.edge(str(source), str(sink), 
                             label=f'Length = {len(edge.seq)}; Coverage = {round(edge.coverage, 2)}')
                
        return dot
        
K = 55
G = DeBruijnGraph(k=K)
# G.read_file('test2.fasta')
 
G.read_file('s_6.first1000.fastq', 'fastq')
# G.read_file('test2.fasta')

G.compress_edges()
G.make_dot_graph()
# G.is_compressed()

KeyError: 'CCACCATTACCACCACCATCACCATTACCACAGGTAACGGTGCGGGCTGACGCGA'

In [ ]:
class DeBruijnGraph:
    def __init__(self, k=55):
        self.k = k
        
    def read_file(self, path, extension='fastq'):
        with open(path, 'r') as f:
            for rec in SeqIO.parse(path, extension):
                

In [262]:
G.adj_list_out

defaultdict(<function __main__.DeBruijnGraph.__init__.<locals>.<lambda>()>,
            {293: defaultdict(set,
                         {294: {Edge(source=293, sink=294, coverage=145.5, seq='GAACGTTTTCTGCGTGTTGCCGATATTCTGGAAAGCAATGCCAGGCAGGGGCAGGT')},
                          1350: {Edge(source=293, sink=1350, coverage=94.5, seq='GAACGTTTTCTGCGTGTTGCCGATATTCTGGAAAGCAATGCCAGGCAGGGGCAGGG')}}),
             294: defaultdict(set,
                         {224: {Edge(source=294, sink=224, coverage=156.06696428571428, seq='AACGTTTTCTGCGTGTTGCCGATATTCTGGAAAGCAATGCCAGGCAGGGGCAGGTGGCCACCGTCCTCTCTGCCCCCGCCAAAATCACCAACCACCTGGTGGCGATGATTGAAAAAACCATTAGCGGCCAGGATGCTTTACCCAATATCAGCGATGCCGAACGTATTTTTGCCGAACTTTTGACGGGACTCGCCGCCGCCCAGCCGGGGTTCCCGCTGGCGCAATTGAAAACTTTCGTCGATCAGGAATTTGCCCAAATAAAACATGTCCTGCATGGC')}}),
             941: defaultdict(set,
                         {942: {Edge(source=941, sink=942, coverage=101.5, seq='CGCATGGTTGTTACCTCGTTACCTTTGGTCGAAAAAAAAAGCCCGCACTGTCAGGT')},
               

In [227]:
ls

ECOLI_IS220_QUAKE_1K_paired_reads.fasta  s_6.first10000.fastq  test2.fasta
ECOLI_IS220_QUAKE_1K_single_reads.fasta  s_6.first1000.fastq
s_6.first100000.fastq                    test1.fasta


In [112]:
len(G.adj_list)

3

In [40]:
ls

ECOLI_IS220_QUAKE_1K_paired_reads.fasta  s_6.first10000.fastq.gz  test2.fasta
ECOLI_IS220_QUAKE_1K_single_reads.fasta  s_6.first1000.fastq.gz
s_6.first100000.fastq.gz                 test1.fasta


In [45]:
k = 55

G = DeBruijnGraph()

G.read_file('test1.fasta')
G.kmer_mapping

...


{Seq('AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGA'): 0,
 Seq('GCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAG'): 1,
 Seq('CTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGT'): 2,
 Seq('TTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTG'): 3,
 Seq('TTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGT'): 4,
 Seq('TTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTC'): 5,
 Seq('TCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCT'): 6,
 Seq('CATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTG'): 7,
 Seq('ATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGA'): 8,
 Seq('TTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGAT'): 9,
 Seq('TCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATA'): 10,
 Seq('CTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAG'): 11,
 Seq('TGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGC'): 12,
 Seq('GACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCA'): 13,
 Seq('ACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAA

In [18]:
print(rec.seq)

AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGCTTCTGAACTGGTTACCTGCCGTGAGTAAATTAAAATTTTATTGACTTAGGTCACTAAATACTTTAACCAATATAGGCATAGCGCACAGACAGATAAAAATTACAGAGTACACAACATCCATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCACAGGTAACGGTGCGGGCTGACGCGTACAGGAAACACAGAAAAAAGCCCGCACCTGACAGTGCGGGCTTTTTTTTTCGACCAAAGGTAACGAGGTAACAACCATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAACGTTTTCTGCGTGTTGCCGATATTCTGGAAAGCAATGCCAGGCAGGGGCAGGTGGCCACCGTCCTCTCTGCCCCCGCCAAAATCACCAACCACCTGGTGGCGATGATTGAAAAAACCATTAGCGGCCAGGATGCTTTACCCAATATCAGCGATGCCGAACGTATTTTTGCCGAACTTTTGACGGGACTCGCCGCCGCCCAGCCGGGGTTCCCGCTGGCGCAATTGAAAACTTTCGTCGATCAGGAATTTGCCCAAATAAAACATGTCCTGCATGGCATTAGTTTGTTGGGGCAGTGCCCGGATAGCATCAACGCTGCGCTGATTTGCCGTGGCGAGAAAATGTCGATCGCCATTATGGCCGGCGTATTAGAAGCGCGCGGTCACAACGTTACTGTTATCGATCCGGTCGAAAAACTGCTGGCAGTGGGGCATTACCTCGAATCTACCGTCGATATTGCTGAGTCCACCCGCCGTATTGCGGCAAGCCGCATTCCGGCTGATCACATGGTGCTGATGGCAGGTTTCACCGCCGGTAATGAAAAAGGCGAACTGGTGGTGCTTGGACGCAACGGTTCCGACTACTCTGCTGCGGTGCTGGCTGCCTGTTTACGCGCCGATT

In [5]:
SeqIO.parse?

In [4]:

SeqIO?